This notebook creates the tables within gold layer

In [0]:
from pyspark.sql.functions import col, year, weekofyear, monotonically_increasing_id, sum, max, try_divide, lag
from pyspark.sql.window import Window

# Country_dim

In [0]:
df_country = spark.table('cat_covid.silver.covid').select('Country_code', 'Country', 'WHO_Region').distinct().orderBy('Country', desc=True)

df_country = df_country.withColumn('ID', 1 + monotonically_increasing_id())

In [0]:
df_country.write.mode('overwrite').format('delta').saveAsTable('cat_covid.gold.country_dim')

# Covid_fact

In [0]:
df = (spark.table('cat_covid.silver.covid')
      .withColumn('Year', year(col('Date_reported')))
      .withColumn('Week', weekofyear(col('Date_reported')))
      )
df_pop = spark.table('cat_covid.silver.population').select('Country_Name', col('Year').alias('Year_pop'), 'Population')

df_country = spark.table('cat_covid.gold.country_dim').select(col('Country_code').alias('country_code_dim'), col('ID').alias('ID_country'))


In [0]:
df = (df
      .join(df_pop, (df.Country == df_pop.Country_Name) & (df.Year == df_pop.Year_pop) , 'left')
      .join(df_country, df.Country_code == df_country.country_code_dim, 'left')
    #   .drop('Country_code', 'Country', 'Country_Name','country_code_dim', 'WHO_Region', 'Year_pop')
      .select(
          col('Date_reported').cast('date'),
          col('New_cases').cast('integer'),
          col('Cumulative_cases').cast('bigint'),
          col('New_deaths').cast('integer'),
          col('Cumulative_deaths').cast('bigint'),
          col('Last_update').cast('date'),
          col('Year').cast('integer'),
          col('Week').cast('integer'),
          col('Population').cast('bigint'),
          col('ID_country').cast('long')
      ))



In [0]:
df = df.groupBy('ID_country','Year','Week').agg(
      max(col('Date_reported')).alias('Date_reported'),
      sum(col('New_cases')).alias('New_cases'),
      max(col('Cumulative_cases')).alias('Cumulative_cases'),
      sum(col('New_deaths')).alias('New_deaths'),
      max(col('Cumulative_deaths')).alias('Cumulative_deaths'),
      max(col('Population')).alias('Population')
)


In [0]:
df = (df
      .withColumn('NewCases_ratio', try_divide(col('New_cases'), col('Population')) * 100000)
      .withColumn('CumCases_ratio', try_divide(col('Cumulative_cases'), col('Population')) * 100000)
      .withColumn('NewDeaths_ratio', try_divide(col('New_deaths'), col('Population')) * 100000)
      .withColumn('CumDeaths_ratio', try_divide(col('Cumulative_deaths'), col('Population')) * 100000)
      .withColumn('Fatality_ratio', try_divide(col('Cumulative_cases'), col('Cumulative_deaths')) )
)

window_spec = Window.partitionBy('ID_country').orderBy('Year', 'Week')
df = (df
      .withColumn('Week1_cases', lag('New_cases',1).over(window_spec))
      .withColumn('Week1_deaths', lag('New_deaths',1).over(window_spec))
)

df = df.fillna(0)

In [0]:
df.write.mode('overwrite').format('delta').option('overwriteSchema', True).saveAsTable('cat_covid.gold.covid')